# Melon 주간 Top 100 구하기

In [37]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [38]:
import requests
url = 'https://www.melon.com/chart/week/index.htm'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
req = requests.get(url, headers = header)
html = req.text

In [39]:
soup = BeautifulSoup(html, 'html.parser')

#### 기간 찾기

In [40]:
soup.select_one('.yyyymmdd').get_text()

'\r\n\t\t\t\t\t\t\t2020.11.09 ~ 2020.11.15\r\n\t\t\t\t\t\t'

In [41]:
date = soup.select_one('.yyyymmdd').get_text().strip()
date

'2020.11.09 ~ 2020.11.15'

In [42]:
start_day = date[:10]
last_day = date[13:]
start_day, last_day

('2020.11.09', '2020.11.15')

In [46]:
service_list = soup.select_one('.service_list_song')
tbody = service_list.find('tbody')
trs = tbody.find_all('tr')
len(trs)

100

### rank 찾기

In [55]:
rank = int(trs[0].select_one('.rank').string)
rank

1

#### 등락 찾기

In [87]:
# 지난 주 실적이 있는 곡: 0면 순위변동 없음, +면 상승, -면 하락
# 지난 주 실적이 없는 곡: 신규 진입(diff = -100) 

for i in [0, 7, 17, 19]:
    entry = trs[i].select_one('.rank_wrap')
    spans = entry.find_all('span')
    diff = -100         # diff = -100 이면 신규 진입
    if len(spans) == 3:
        diff = int(spans[2].string)
        # 지난 주 대비 순위 변동 있을 시
        if diff != 0:
            sign = spans[2].attrs['class']
            if sign[0] == 'down':
                diff *= -1
    print(diff)

0
3
-5
-100


In [69]:
# 지난 주 실적이 있는 곡 - 지난 주 대비 순위 동일
span = trs[0].select_one('.rank_wrap')
span.find_all('span')

[<span class="bullet_icons rank_static"><span class="none">순위 동일</span></span>,
 <span class="none">순위 동일</span>,
 <span class="none">0</span>]

In [84]:
# 지난 주 실적이 있는 곡 - 지난 주 대비 3단계 상승
span = trs[7].select_one('.rank_wrap')
span.find_all('span')

[<span class="bullet_icons rank_up"><span class="none">단계 상승</span></span>,
 <span class="none">단계 상승</span>,
 <span class="up">3</span>]

In [68]:
# 지난 주 실적이 있는 곡 - 지난 주 대비 1단계 하락
span = trs[2].select_one('.rank_wrap')
span.find_all('span')

[<span class="bullet_icons rank_down"><span class="none">단계 하락</span></span>,
 <span class="none">단계 하락</span>,
 <span class="down">1</span>]

In [70]:
# 지난 주 실적이 없는 곡 - 신규 진입
span = trs[19].select_one('.rank_wrap')
span.find_all('span')

[<span class="bullet_icons rank_new"><span class="none">순위 진입</span></span>,
 <span class="none">순위 진입</span>]

#### 노래 제목, 가수 찾기

In [78]:
info = trs[0].select('.ellipsis')
title = info[0].find('a').string
singer = info[1].find('a').string
album = info[2].find('a').string
title, singer, album

('Dynamite', '방탄소년단', 'Dynamite (DayTime Version)')

### 데이터프레임으로 만들기

In [109]:
ranks = []; diffs = []; titles = []; singers = []; albums = []
service_list = soup.select_one('.service_list_song')
tbody = service_list.find('tbody')
trs = tbody.find_all('tr')

In [110]:
for i, tr in enumerate(trs):
    rank = int(tr.select_one('.rank').string)

    info = tr.select_one('.ellipsis.rank01')
    title = info.find('a').string
    info = tr.select_one('.ellipsis.rank02')
    singer = info.find('a').string
    info = tr.select_one('.ellipsis.rank03')
    album = info.find('a').string

    entry = tr.select_one('.rank_wrap')
    spans = entry.find_all('span')
    diff = -100         # diff = -100 이면 신규 진입
    if len(spans) == 3:
        diff = int(spans[2].string)
        # 지난 주 대비 순위 변동 있을 시
        if diff != 0:
            sign = spans[2].attrs['class']
            if sign[0] == 'down':
                diff *= -1

    ranks.append(rank); diffs.append(diff)
    titles.append(title); singers.append(singer)
    albums.append(album)

In [111]:
top100 = pd.DataFrame({
    'rank': ranks,
    'diff': diffs,
    'title': titles,
    'singer': singers,
    'album': albums
})
top100 = top100.set_index('rank')
print(f'기간: {start_day} ~ {last_day}')
top100.head(20)

기간: 2020.11.09 ~ 2020.11.15


,diff,title,singer,album
rank,,,,
1,0,Dynamite,방탄소년단,Dynamite (DayTime Version)
2,1,힘든 건 사랑이 아니다,임창정,힘든 건 사랑이 아니다
3,-1,Lovesick Girls,BLACKPINK,THE ALBUM
4,1,취기를 빌려 (취향저격 그녀 X 산들),산들,취기를 빌려 (취향저격 그녀 X 산들)
5,-1,DON'T TOUCH ME,환불원정대,DON'T TOUCH ME
6,0,잠이 오질 않네요,장범준,잠이 오질 않네요
7,0,오래된 노래,스탠딩 에그,오래된 노래
8,3,딩가딩가 (Dingga),마마무 (Mamamoo),딩가딩가 (Dingga)
9,-1,Savage Love (Laxed - Siren Beat) (BTS Remix),Jawsh 685,Savage Love (Laxed - Siren Beat) [BTS Remix]
